In [4]:
import json

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [ ]:
def obter_temperatura_atual(local, unidade="celsius"):
    # Função que retorna a temperatura de uma cidade com base no nome da cidade (local)
    # e na unidade de medida (celsius ou fahrenheit). A temperatura é simulada.
    
    if "são paulo" in local.lower():
        # Se o local for "São Paulo" (insensível a maiúsculas/minúsculas), retorna uma
        # temperatura simulada de 32 graus na unidade escolhida.
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
        )
    elif "porto alegre" in local.lower():
        # Se o local for "Porto Alegre", retorna uma temperatura simulada de 25 graus.
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
        )
    elif "rio de janeiro" in local.lower():
        # Se o local for "Rio de Janeiro", retorna uma temperatura simulada de 35 graus.
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
        )
    else:
        # Para qualquer outra cidade, retorna "unknown" como temperatura,
        # indicando que a temperatura não está disponível.
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
        )


# Lista de ferramentas (tools) disponíveis para o modelo usar, definindo uma função
tools = [
    {
        "type": "function",  # Indica que o tipo de ferramenta é uma função
        "function": {
            "name": "obter_temperatura_atual",  # Nome da função que será usada
            "description": "Obtém a temperatura atual em uma dada cidade",  # Descrição da função
            "parameters": {  # Define os parâmetros que a função precisa para funcionar
                "type": "object",  # O tipo de parâmetro é um objeto (JSON)
                "properties": {
                    "local": {  # Primeiro parâmetro, o nome da cidade
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",  # Explica que a função precisa do nome da cidade
                    },
                    "unidade": {  # Segundo parâmetro, a unidade de temperatura
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]  # A unidade pode ser celsius ou fahrenheit
                    },
                },
                "required": ["local"],  # O parâmetro "local" é obrigatório
            },
        },
    }
]

# Dicionário que relaciona os nomes das funções disponíveis com suas implementações
funcoes_disponiveis = {
    "obter_temperatura_atual": obter_temperatura_atual,  # Relaciona o nome da função com a função definida anteriormente
}

# Lista de mensagens, que simula uma interação entre o usuário e o assistente
mensagens = [
    {
        "role": "user",  # Papel do usuário no chat
        "content": "Qual é a temperatura em São Paulo, Porto Alegre e Brasília?"  # Pergunta do usuário
    }
]

# Chama a API da OpenAI para gerar uma resposta com base no modelo especificado
resposta = client.chat.completions.create(
    model="gpt-4o-mini",  # Modelo utilizado para gerar a resposta
    messages=mensagens,  # Mensagens trocadas até o momento
    tools=tools,  # Ferramentas disponíveis para o modelo usar
    tool_choice="auto",  # O modelo escolhe automaticamente a ferramenta a ser usada
)

# Extrai a mensagem gerada pelo modelo
mensagem_resp = resposta.choices[0].message
# Extrai as chamadas de ferramentas feitas pelo modelo, se houver
tool_calls = mensagem_resp.tool_calls

if tool_calls:
    # Se houver chamadas de ferramentas, adiciona a resposta à lista de mensagens
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        # Itera sobre as chamadas de ferramentas para executar as funções associadas
        function_name = tool_call.function.name  # Extrai o nome da função chamada
        function_to_call = funcoes_disponiveis[function_name]  # Pega a função correspondente
        function_args = json.loads(tool_call.function.arguments)  # Converte os argumentos da função de JSON para um dicionário
        # Chama a função com os argumentos fornecidos pelo modelo
        function_response = function_to_call(
            local=function_args.get("local"),  # Obtém o parâmetro 'local' da função
            unidade=function_args.get("unidade"),  # Obtém o parâmetro 'unidade' da função
        )
        # Adiciona a resposta da ferramenta (função) à lista de mensagens
        mensagens.append(
            {
                "tool_call_id": tool_call.id,  # Identificador da chamada da ferramenta
                "role": "tool",  # Papel como ferramenta (função)
                "name": function_name,  # Nome da função chamada
                "content": function_response,  # Resultado da execução da função
            }
        )
    # Após as chamadas de ferramentas, gera uma nova resposta com base nas mensagens atualizadas
    segunda_resposta = client.chat.completions.create(
        model="gpt-4o-mini",  # Modelo utilizado para gerar a segunda resposta
        messages=mensagens,  # Mensagens atualizadas com as respostas das funções
    )

# Extrai a resposta final gerada pelo modelo
mensagem_resp = segunda_resposta.choices[0].message
# Imprime o conteúdo da resposta final
print(mensagem_resp.content)


#### Explicação:

* O script define uma função chamada `obter_temperatura_atual`, que simula a obtenção de temperaturas para algumas cidades específicas.

* Um conjunto de ferramentas (funções) é disponibilizado para um modelo de linguagem da OpenAI. Essas ferramentas podem ser chamadas automaticamente pelo modelo, dependendo do que o usuário perguntar.

* O script utiliza a API da OpenAI para processar uma interação do usuário, onde o modelo tenta responder à pergunta sobre a temperatura em São Paulo e Porto Alegre. Se necessário, o modelo chama as funções definidas para fornecer uma resposta mais precisa.

* Após o modelo utilizar as ferramentas e atualizar as mensagens, ele gera uma resposta final, que é exibida no console.

---

Vou explicar passo a passo o que ocorre nesse script:

1. **Definição da Função `obter_temperatura_atual`**:
   - A função `obter_temperatura_atual` é responsável por retornar a temperatura simulada de uma cidade com base no nome do local passado como parâmetro e na unidade de medida (`celsius` ou `fahrenheit`).
   - Para as cidades "São Paulo", "Porto Alegre" e "Rio de Janeiro", retorna-se uma temperatura fixa simulada (32°C, 25°C e 35°C, respectivamente). Para qualquer outra cidade, retorna "unknown", indicando que a temperatura não está disponível.

2. **Bloco `tools`**:
   - O script define uma lista chamada `tools`, que contém informações sobre as ferramentas disponíveis para o modelo de linguagem usar. Aqui, a ferramenta é uma função chamada `obter_temperatura_atual`.
   - A estrutura indica que a função requer dois parâmetros: `local` (nome da cidade) e `unidade` (unidade de temperatura). O parâmetro `local` é obrigatório.

3. **Dicionário `funcoes_disponiveis`**:
   - Este dicionário relaciona o nome da função (`obter_temperatura_atual`) com sua implementação, que é a própria função definida anteriormente. Esse dicionário será usado posteriormente para chamar a função correspondente.

4. **Bloco `mensagens`**:
   - A lista `mensagens` simula uma troca de mensagens em um contexto de chat, onde o usuário pergunta "Qual é a temperatura em São Paulo e Porto Alegre?". Essa mensagem será passada ao modelo.

5. **Chamada da API do modelo**:
   - O script faz uma chamada à API da OpenAI, especificando o modelo a ser utilizado (`gpt-4o-mini`) para gerar uma resposta com base nas mensagens anteriores.
   - As ferramentas disponíveis (`tools`) são fornecidas para que o modelo possa escolher qual ferramenta usar.
   - A opção `"tool_choice": "auto"` permite que o modelo selecione automaticamente a ferramenta apropriada para responder à pergunta do usuário.

6. **Extração da resposta gerada**:
   - A resposta gerada pelo modelo é armazenada na variável `mensagem_resp`.
   - Em seguida, são verificadas se há chamadas de ferramentas (funções) na resposta. Se houver, o código irá lidar com elas.

7. **Execução das funções chamadas**:
   - Se o modelo solicitou a execução de alguma ferramenta (função), o código entra em um loop que percorre todas as chamadas de funções realizadas pelo modelo.
   - Para cada chamada de função, ele extrai o nome da função e os argumentos fornecidos (que estão em formato JSON).
   - A função correspondente é chamada com os argumentos fornecidos, e o resultado dessa função é adicionado à lista de mensagens.

8. **Nova resposta após a execução das funções**:
   - Após as funções serem executadas, o script faz uma nova chamada à API da OpenAI, desta vez utilizando o modelo `gpt-3.5-turbo-0125`, e inclui as novas mensagens com os resultados das funções.
   - A resposta final do modelo é extraída e impressa.

Em resumo, o código simula uma conversa onde o usuário pergunta sobre a temperatura em São Paulo e Porto Alegre, o modelo de linguagem escolhe e chama a função `obter_temperatura_atual`, que retorna as temperaturas simuladas dessas cidades. Em seguida, a resposta final é impressa com base nas informações retornadas pelas funções.